<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/main/Class_03_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

**Module 3: Introduction to TensorFlow**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Integrative Biology](https://sciences.utsa.edu/integrative-biology/), [UTSA](https://www.utsa.edu/)


### Module 3 Material

* Part 3.1: Deep Learning and Neural Network Introduction
* Part 3.2: Using Keras to Build Regression Models
* **Part 3.3: Using Keras to Build Classification Models**
* Part 3.4: Saving and Loading a Keras Neural Network
* Part 3.5: Early Stopping in Keras to Prevent Overfitting

### Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [ ]:
# YOU MUST RUN THIS CELL FIRST

try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    from google.colab import auth
    auth.authenticate_user()
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
    import requests
    gcloud_token = !gcloud auth print-access-token
    gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
    print(gcloud_tokeninfo['email'])
except:
    print("Note: not using Google CoLab")
    COLAB = False

**Neural Network Classification and Regression**
![Neural Network Classification and Regression](https://biologicslab.co/BIO1173/images/class_2_ann_class_reg.png "Neural Network Classification and Regression")

# Part 3.3: Using Keras to Build Classification Models

In the previous lesson (Class_03_2) we looked at how to use Keras/Tensorflow to build simple neural network models that performed a regression analysis. In this lesson, we continue our introduction into Keras and Tensorflow by building neural network models that perform a **_classification_**.

**_Classification_** is how a neural network attempts to classify the input into one or more **_classes_**.  The simplest way of evaluating a classification network is to track the percentage of training set items classified incorrectly.  

We typically score human results in this manner.  For example, you might have taken multiple-choice exams in school in which you had to shade in a bubble for choices A, B, C, or D.  If you chose the wrong letter on a 10-question exam, you would earn a 90%.  In the same way, we can grade computers; however, most classification algorithms do not merely choose A, B, C, or D.  Computers typically report a classification as their percent confidence in each class.  The figure below shows how a computer and a human might respond to question number 1 on an exam.

**Classification Neural Network Output**
![Classification Neural Network Output](https://biologicslab.co/BIO1173/images/class-multi-choice.png)

As you can see, the human test taker marked the first question as "B." However, the computer test taker had an 80% (0.8) confidence in "B" and was also somewhat sure with 10% (0.1) on "A." The computer then distributed the remaining points to the other two.  

In the simplest sense, the machine would get 80% of the score for this question if the correct answer were "B." The computer would get only 5% (0.05) of the points if the correct answer were "D."

We previously saw how to train a neural network to predict either the `Ripeness` or the `Acidity` of an apple. In this lesson, we will now see how to build a neural network to predict a **_class_**. In particular, we are going to build a neural network that can predict the **_species_** of a particular Iris flower based either on it's sepal dimensions or on its petal dimensions.

The code to construct a neural network that can classify Iris flowers is similar to the code for building a neural network for regression. However, there are 3 important differences in classification models:

* The output neuron count is **not 1**, but matches the number of classes.
* The **_Softmax_** transfer function is utilized by the output layer.
* The loss function is **_cross entropy_**.
  

## Example 1: Predict Iris Species

In Example 1, our goal will be to predict the `class` (species) of an _Iris_ flower based _only_ on the length and width of its **_sepals_**. In **Example 1** you will repeat the analysis, but your model will only use the **_petal_** measurements for training. Although sepals and petals in Iris flowers look similar, they are distinctly different if you look closely.

The next image shows an example of how sepal and petal measurements are made.

**Iris Sepal and Petal Measurements**

![____](https://biologicslab.co/BIO1173/images/iris_petal_sepal.png)

As was done in Lesson_03_3, the code for Example 1 will be divided into the following 3 steps:

1. Read datafile and create DataFrame
2. Create feature vector
3. Construct, compile and train the neural network model

When constructing neural networks, AI researchers typically follow the same 3 steps.

### Example 1-Step 1: Read datafile and create DataFrame

The first step is usually to either read the a dataset from a local file, or download one from the Internet. In this example, we will read the Iris Flower dataset stored in a CSV file called `iris.csv` on the course HTTPS server, `https://biologicslab.co`, using this code chunk:
~~~text
# Read dataset into a DataFrame
sepalDF = pd.read_csv(
    "http://biologicslab.co/BIO1173/data/iris.csv",
            na_values=['NA', '?'])
~~~
As the dataset is being read, it is being stored in a Pandas DataFrame called `sepalDF`. This name was choosen to remind us that this DataFrame is being used for Example 1. In this example, we are going to **_shuffle_** and reindex the data after setting the random seed = 42.

Finally, after creating our new DataFrame and shuffling the data, we display the first few rows and columns to make sure the data was read correctly.

In [2]:
# Example 1: Read datafile and create DataFrame

import pandas as pd
import numpy as np

# Read dataset into a DataFrame
sepalDF = pd.read_csv(
    "http://biologicslab.co/BIO1173/data/iris.csv",
            na_values=['NA', '?'])

# Set the random seed to 42
np.random.seed(42)

# Shuffle & reindex
sepalDF = sepalDF.reindex(np.random.permutation(sepalDF.index))

# Set the max rows and max columns
pd.set_option('display.max_rows', 8)
pd.set_option('display.max_columns', 8)

# Display the DataFrame
display(sepalDF)


,sepal_length,sepal_width,petal_length,petal_width,species
73,6.1,2.8,4.7,1.2,Iris-versicolor
18,5.7,3.8,1.7,0.3,Iris-setosa
118,7.7,2.6,6.9,2.3,Iris-virginica
78,6.0,2.9,4.5,1.5,Iris-versicolor
...,...,...,...,...,...
106,4.9,2.5,4.5,1.7,Iris-virginica
14,5.8,4.0,1.2,0.2,Iris-setosa
92,5.8,2.6,4.0,1.2,Iris-versicolor
102,7.1,3.0,5.9,2.1,Iris-virginica


If your code is correct, you should see the following table:

![___](https://biologicslab.co/BIO1173/images/class_03_3_Shuffle.png)


As you can see, all of the information in the DataFrame is numeric, with the exception of the column `species`. We need to keep this mind when we create our feature vector in the next step.

### Example 1-Step 2: Create Feature Vector

While we normally begin by converting all the string values to integers, we are **_not_** going to do this in this particular example. The only column with non-numeric values is the column `species`. Since the `species` column will be our Y-values, we take care of it **_after_** we first generate our X-values.

Normally we would like to use **all** of the useful data that is available in a dataset for training our neural network. However, since this is a teaching example, we will limit our X-values to just the sepal measurements. We can limit what columns in our DataFrame are used for generating the X-values by simply identifying the specific columns that we want to use by name, as shown in the following code chunk:
~~~text
# Generate X-values
sepalX = sepalDF[['sepal_length', 'sepal_width']].values
sepalX = np.asarray(sepalX).astype('float32')
~~~
When generate either X or Y values, we always want to make sure that they are type `float32` by using the code line:
~~~text
sepalX = np.asarray(sepalX).astype('float32')
~~~

One **critical difference** between classification neural networks and regression neural networks is how the Y-values are generated. In regression, you can directly use the numerical values in the correct column in the DataFrame as was demonstrated in the previous lesson. However, when creating a feature vector for a classification neural network, **_the Y-values must be One-Hot Encoded!_**

Here is the code chunk for generating the Y-values for classification using One-Hot Encoding:
~~~text
# Generate Y-values
dummies = pd.get_dummies(sepalDF['species'], dtype=int) # Classification
iris_species = dummies.columns
sepalY = dummies.values
sepalY = np.asarray(sepalY).astype('float32')
~~~

To verify that our coding was correct, the example prints the first 10 values in the variable `dummies` and our Numpy array containing the Y-values, `sepalY`.


In [3]:
# Example 1-Step 2: Create Feature Vector

import pandas as pd
import numpy as np

# Generate X-values
sepalX = sepalDF[['sepal_length', 'sepal_width']].values
sepalX = np.asarray(sepalX).astype('float32')

# Generate Y-values
dummies = pd.get_dummies(sepalDF['species'], dtype=int) # Classification
iris_species = dummies.columns
sepalY = dummies.values
sepalY = np.asarray(sepalY).astype('float32')

# Print dummies
print(dummies[0:10])

# Print first 10 Y-values
print(sepalY[0:10])


     Iris-setosa  Iris-versicolor  Iris-virginica
73             0                1               0
18             1                0               0
118            0                0               1
78             0                1               0
..           ...              ...             ...
64             0                1               0
141            0                0               1
68             0                1               0
82             0                1               0

[10 rows x 3 columns]
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


If your code is correct you should see the following output:
~~~text
     Iris-setosa  Iris-versicolor  Iris-virginica
73             0                1               0
18             1                0               0
118            0                0               1
78             0                1               0
..           ...              ...             ...
64             0                1               0
141            0                0               1
68             0                1               0
82             0                1               0

[10 rows x 3 columns]
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
~~~

The variable called `dummies` is actually a DataFrame that was generated as part of the One-Hot Encoding. The first 10 records in the `dummies` DataFrame are shown at the top of this output. You should note that the `dummies` DataFrame has **_exactly_** the same numeric values as the Numpy array `sepalY` shown at the bottom of the output. This makes sense given that `sepalY` was generated by the following code chunk:
~~~text
sepalY = dummies.values
~~~
This line of code simply converts the numeric **_values_** in the `dummies` DataFrame into a Numpy array, `sepalY`.

For example, the first flower in `dummies` (index number `73`) has a `1` in the second column `Iris-versicolor`. If you look at the first element in the `sepalY` array, there is also a `1` in the second column.

### Example 1-Step 3: Construct, Compile and Train Neural Network

As mentioned above, the neural network that we will use to classify Iris flowers based on their sepal dimensions, is very similar to a regression neural network but with the following exceptions:

* The output neuron count matches the number of classes (in this case 3, since there are 3 different species).
* The `Softmax` transfer function is utilized by the output layer.
* The loss function is `cross entropy`.

The code in the cell below builds a simple neural network called `sepalModel`. Notice that the number of neurons in the input layer is specified by `input_dim=sepalX.shape[1]`:
~~~text
sepalModel.add(Dense(50, input_dim=sepalX.shape[1],
                     activation='relu')) # Hidden 1
~~~
Also the output layer of this classification neural network is different:
~~~text
sepalModel.add(Dense(sepalY.shape[1],activation='softmax')) # Output
~~~
First, the are more than 1 neuron in the output layer. In a classification neural network, there must be 1 neuron for each class. Since there are 3 classes (species) in the Iris Flower dataset, there has to be 3 output neurons. The number of output neurons is specified by the argument `sepalY.shape[1]`.

The other difference in the output layer is the presence of an activation function. In this example, the activation function is `softmax`. In a regression neural network, the single output neuron does **not** have an activation function.

Finally, you should notice that when the model is compiled, it uses the `categorical_crossentropy` loss function instead of the `mean_squared_error` loss function used in regression neural networks.

In [6]:
# Example 1-Step 2: Construct, compile and train neural network

import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import Adam

# Construct model
sepalModel = Sequential()
sepalModel.add(Input(shape=(sepalX.shape[1],))) # Hidden 1
sepalModel.add(Dense(25, activation='relu')) # Hidden 2
sepalModel.add(Dense(sepalY.shape[1],activation='softmax')) # Output

# Compile the model
sepalModel.compile(loss='categorical_crossentropy', optimizer='adam')

# Print model summary (optional)
sepalModel.summary()

# Train model
sepalModel.fit(sepalX,sepalY,verbose=2,epochs=100)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 25)                  │              75 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │              78 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 153 (612.00 B)

 Trainable params: 153 (612.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
5/5 - 1s - 182ms/step - loss: 2.2706
Epoch 2/100
5/5 - 0s - 7ms/step - loss: 2.1261
Epoch 3/100
5/5 - 0s - 7ms/step - loss: 2.0115
Epoch 4/100
5/5 - 0s - 7ms/step - loss: 1.9095
Epoch 5/100
5/5 - 0s - 6ms/step - loss: 1.8233
Epoch 6/100
5/5 - 0s - 6ms/step - loss: 1.7339
Epoch 7/100
5/5 - 0s - 6ms/step - loss: 1.6512
Epoch 8/100
5/5 - 0s - 6ms/step - loss: 1.5714
Epoch 9/100
5/5 - 0s - 7ms/step - loss: 1.4879
Epoch 10/100
5/5 - 0s - 6ms/step - loss: 1.4151
Epoch 11/100
5/5 - 0s - 6ms/step - loss: 1.3469
Epoch 12/100
5/5 - 0s - 6ms/step - loss: 1.2892
Epoch 13/100
5/5 - 0s - 6ms/step - loss: 1.2381
Epoch 14/100
5/5 - 0s - 7ms/step - loss: 1.1910
Epoch 15/100
5/5 - 0s - 6ms/step - loss: 1.1574
Epoch 16/100
5/5 - 0s - 6ms/step - loss: 1.1291
Epoch 17/100
5/5 - 0s - 7ms/step - loss: 1.1121
Epoch 18/100
5/5 - 0s - 6ms/step - loss: 1.0949
Epoch 19/100
5/5 - 0s - 6ms/step - loss: 1.0851
Epoch 20/100
5/5 - 0s - 6ms/step - loss: 1.0806
Epoch 21/100
5/5 - 0s - 6ms/step - loss: 1.0745

If your code is correct you should see the following output:

~~~text
Model: "sequential_2"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 25)                  │              75 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │              78 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘
 Total params: 153 (612.00 B)
 Trainable params: 153 (612.00 B)
 Non-trainable params: 0 (0.00 B)
Epoch 1/100
5/5 - 1s - 182ms/step - loss: 2.2706
Epoch 2/100
5/5 - 0s - 7ms/step - loss: 2.1261
Epoch 3/100
5/5 - 0s - 7ms/step - loss: 2.0115
Epoch 4/100
5/5 - 0s - 7ms/step - loss: 1.9095
Epoch 5/100
5/5 - 0s - 6ms/step - loss: 1.8233p
..................

Epoch 95/100
5/5 - 0s - 11ms/step - loss: 0.8004
Epoch 96/100
5/5 - 0s - 6ms/step - loss: 0.7969
Epoch 97/100
5/5 - 0s - 6ms/step - loss: 0.7937
Epoch 98/100
5/5 - 0s - 6ms/step - loss: 0.7903
Epoch 99/100
5/5 - 0s - 6ms/step - loss: 0.7877
Epoch 100/100
5/5 - 0s - 11ms/step - loss: 0.7844
<keras.src.callbacks.history.History at 0x79bfedba82e0>
~~~

### Example 2: Print Predictions

Now that we have trained our neural network, `sepalModel`, we would like to be able to use it. As before, we will generate predictions. Instead of given us a single prediction, as the regression neural network model did in Class_03_2, our new model will make **_3 predictions_**, one prediction for each **_class_** in the dependent variable. These three predictions represents the 3 probabilities that a particular unknown flower is (1) _Iris setosa_, (2) _Iris versicolor_, and (3) _Iris virginica_.

In [7]:
# Example 2: Print Predictions

import numpy as np

# Compute the model predictions
sepalPred = sepalModel.predict(sepalX)

# Change print from scientific notation
np.set_printoptions(suppress=True)

# Print out the results
print(f"Shape of sepalPred: {sepalPred.shape}")
print(sepalPred[0:10])

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Shape of sepalPred: (150, 3)
[[0.17597489 0.413231   0.41079417]
 [0.61388004 0.13897833 0.24714172]
 [0.07184878 0.5347419  0.39340925]
 [0.21679893 0.37867028 0.4045308 ]
 [0.10337108 0.4839621  0.41266686]
 [0.52072614 0.18672682 0.29254714]
 [0.2850867  0.3292407  0.38567263]
 [0.14490539 0.4368158  0.4182788 ]
 [0.10122577 0.4995702  0.39920396]
 [0.19212958 0.40105787 0.40681252]]


If your code is correct your should see something similar to the following:

~~~text
5/5 [==============================] - 0s 2ms/step
Shape of sepalPred: (150, 3)
[[0.00667937 0.49824646 0.49507418]
 [0.9701563  0.02258347 0.00726027]
 [0.00000133 0.2490988  0.75089985]
 [0.02025999 0.5275467  0.4521932 ]
 [0.00030747 0.40100244 0.59869015]
 [0.8894263  0.07975859 0.0308151 ]
 [0.10499485 0.5329848  0.3620204 ]
 [0.00158804 0.4518695  0.5465425 ]
 [0.0000668  0.3567878  0.6431454 ]
 [0.01239389 0.51633203 0.47127402]]
~~~

Each row of numbers represents the model's prediction for one Iris flower in the dataset. The first column represents the **_probability_** that the flower's species is _I. setosa_, the second column is the probability for _I. versicolor_, and the third column is the probability for _I. virginica_.

You should note two things about these predictions. First, generally, one column has a significantly higher probability than the other two columns, but this is not always the case.

In the particular example shown above, there is a very low probability that the first flower is _I. setosa_ (prob=0.0067), but the `sepalModel` assigned essentially equal probabilites to the flowere being  _I. versicolor_ (prob=0.498 or 49.8%) and _I. virginica_ (prob=0.495 or 49.5%). In other words, our `sepalModel` thinks there is a 50=50 chance that this particular flower is either  _I. versicolor_ or _I. virginica_.

For the next flower on the list, our `sepalModel` is very confident (prob=0.970 or 97%) that it is  _I. setosa_. Finally, `sepalModel` is moderately confident (prob=0.750 or 75%) that the third flower is _I. virginica_ but, the is some possibilty (prob=0.249 or 25%) that it is really _I. virginica_.  

As you might expect, when you add up all three probabilities for any particular flower, the sum is essentially zero. To demonstrate this, the next code cell adds the three probabilites of the last flower in the list.

In [ ]:
# Add up the probabilites for list flower in the list
0.01239389 + 0.51633203 + 0.47127402

0.9999999400000001

As you can see, it's pretty close to 1. That makes sense since there is a 100% probability that any flower will be one of the three possible species in the dataset.

### Example 3: Print Predicted and Expected Values

Usually, the program considers the column with the **_highest_** prediction to be the prediction of the neural network. The `np.argmax()` function can be used to find the index of the **_maximum prediction_** for each row.

As shown in the cell below, we can use `np.argmax()` function generate the variables `sepalPredict_classes` and `sepalExpected_classes`. We will need these variables shortly when we want to compute the model's accuracy score.


In [8]:
# Example 3: Print Predicted and Expected Values

import numpy as np

# Find the maximum prediction for each row
sepalPredict_classes = np.argmax(sepalPred,axis=1)

# Find the expected value for each row
sepalExpected_classes = np.argmax(sepalY,axis=1)

# Print out the results
print(f"Predictions: {sepalPredict_classes}")
print(f"Expected: {sepalExpected_classes}")

Predictions: [1 0 1 2 1 0 2 1 1 2 2 0 0 0 0 2 1 1 2 1 0 2 0 1 2 1 1 2 0 0 0 0 2 0 0 1 2
 0 0 0 2 0 2 0 0 1 1 2 1 2 2 1 2 0 1 2 0 0 0 1 1 0 0 0 2 0 2 2 0 2 1 0 2 2
 1 0 2 1 0 2 1 0 0 1 1 0 2 0 0 2 2 2 1 1 1 1 0 0 1 1 0 0 0 1 2 0 1 2 0 2 1
 1 1 2 0 1 2 1 1 2 2 2 1 2 0 2 1 2 0 2 2 1 0 2 0 1 1 1 2 1 1 1 2 2 0 1 2 0
 1 1]
Expected: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 2 2 1 2 1 2 1 0 2 1 0 0 0 1 2 0 0 0 1 0 1 2 0 1 2 0 2 2
 1 1 2 1 0 1 2 0 0 1 1 0 2 0 0 1 1 2 1 2 2 1 0 0 2 2 0 0 0 1 2 0 2 2 0 1 1
 2 1 2 0 2 1 2 1 1 1 0 1 1 0 1 2 2 0 1 2 2 0 2 0 1 2 2 1 2 1 1 2 2 0 1 2 0
 1 2]


If your code is correct you should see something similar to the following:

~~~text
Predictions: [2 0 2 1 2 0 1 2 2 1 2 0 0 0 0 1 2 2 1 2 0 1 0 2 2 2 2 2 0 0 0 0 2 0 0 2 1
 0 0 0 1 1 2 0 0 2 2 2 2 2 1 2 1 0 2 1 0 0 0 2 2 0 0 0 1 0 1 2 0 2 2 0 1 1
 2 1 1 2 0 1 2 0 0 2 2 0 1 0 0 1 1 2 2 2 2 1 0 0 2 2 0 0 0 2 1 0 2 1 0 2 2
 2 2 1 0 2 1 2 2 1 1 1 2 1 0 1 2 2 0 1 2 2 0 2 0 2 2 2 1 2 2 2 1 1 0 2 1 0
 2 2]
Expected: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0 1 2 2 1 2 1 2 1 0 2 1 0 0 0 1 2 0 0 0 1 0 1 2 0 1 2 0 2 2
 1 1 2 1 0 1 2 0 0 1 1 0 2 0 0 1 1 2 1 2 2 1 0 0 2 2 0 0 0 1 2 0 2 2 0 1 1
 2 1 2 0 2 1 2 1 1 1 0 1 1 0 1 2 2 0 1 2 2 0 2 0 1 2 2 1 2 1 1 2 2 0 1 2 0
 1 2]
~~~

### Example 4 - Convert Index Values into Species Names

With a little bit of python coding, it's not too hard to change the numbers `0`, `1` and `2` into their corresponding species names. The trick is the variable `iris_species` which is Python type called `index`. This variable was created as part of the One-Hot Encoding of the `species` column during the creation of the feature vector in Example 1-Step 2.

If you print out `iris_species` you get the following:
~~~text
Index(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype='object')
~~~~
The code in the next cell shows how you can use this `index` type to convert index values in the variable `sepalPred` to their species names.

In [9]:
# Example 4: Convert index values into species names

# Print index values for first 5 flowers
print(f"sepalPredict_classes: {sepalPredict_classes[0:4]}")

# Convert index values to species names
print(*iris_species[sepalPredict_classes[0:4]])

sepalPredict_classes: [1 0 1 2]
Iris-versicolor Iris-setosa Iris-versicolor Iris-virginica


If your code is correct you should see:
~~~text
sepalPredict_classes: [1 0 2 1]
Iris-versicolor Iris-setosa Iris-virginica Iris-versicolor
~~~
As you can see, the code in the cell above converted the 4 index values `[1 0 2 1]` into the 4 species names `Iris-versicolor`,`Iris-setosa`, `Iris-virginica`, and `Iris-versicolor`.

### Example 5: Compute the Accuracy Score

When using a regression neural network, the convention is to use Root Mean Squared Error (RMSE) as a meaure of the model's ability to correctly predict the Y-values (see Class_03_2). As we dicussed previously, interpreting how good a model is by it's RSME is tricky.

The situation is mucn better when working with classification neural networks. Instead of measuring performance with RSME, we use an **_Accuracy Score_** which is a more easily understood error metric. An accuracy score is essentially a test score. As an example, for all of the species predictions made by the `sepalModel`, what percent were correct? In other words, the accuracy score is like the number of correct answers on an exam.  

Unfortunatley, the accuracy score isn't a perfect error metric because it doesn't consider how confident the neural network was in each prediction.

The code in the cell below uses the `accuracy_score()` function imported from the **scikit-learn** package (nickname `sklearn`), to compute the accuracy of the predicitions made by `sepalModel` and stores this value in a variable called `sepalCorrect`.

In [11]:
# Example 5: Compute the accuracy score

from sklearn.metrics import accuracy_score

# Compute accuracy
sepalCorrect = accuracy_score(sepalExpected_classes,sepalPredict_classes)

# Print out the results
print(f"Accuracy: {sepalCorrect}")

Accuracy: 0.6266666666666667


If your code is correct, you should see something similar to the following output:
~~~text
Accuracy: 0.6266666666666667
~~~
The accuracy of the `sepalModel` is roughly 60% accurate.

### Example 6: Use Model to Make an _ad hoc_ Prediction

The code below performs an _ad hoc_ prediction. Suppose we measure the sepal length and width of flower from an unknown Iris species. We can "feed" this information into our trained neural network model `sepalModel` and ask it to predict which species did the flower came from. This is an example of an _ad hoc_ prediction.

The first step is to define a variable `sample_flower` with the measurements (in cm) for the sepal length (6.6) and the sepal width (2.9):
~~~text
sample_flower = np.array( [[6.6,2.9]], dtype=float)
~~~

Since the model `sepalModel` was trained on data where the sepal length was "first" and sepal width was "second", it is essential to submit the sepal measurements in exactly the same order.

The next line of code uses the `sepalModel` to make the actual prediction which is stored in the variable `sepalPred`:
~~~text
sepalPred = sepalModel.predict(sample_flower)
~~~

In [12]:
# Example 6: Use model to make an ad hoc prediction

# Specify the sepal length and width for an unknown Iris flower
sample_flower = np.array( [[6.6,2.9]], dtype=float)

# Use the neural network to predict the species
sepalPred = sepalModel.predict(sample_flower)

# Print out the results
print(sepalPred)
sepalPred = np.argmax(sepalPred)
print(f"Model predicts sepal dimensions {sample_flower} are from: {iris_species[sepalPred]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
[[0.13752858 0.44678456 0.41568688]]
Model predicts sepal dimensions [[6.6 2.9]] are from: Iris-versicolor


If your code is correct you should see the following output:
~~~text
1/1 [==============================] - 0s 20ms/step
[[0.0014901  0.45076337 0.5477465 ]]
Model predicts sepal dimensions [[6.6 2.9]] are from: Iris-virginica
~~~
While our `sepalModel` predicted that the unknown flower was Iris-virginica, by looking at the output, we can see that it isn't very confident with this prediction. Look at the line:
~~~text
[[0.0014901  0.45076337 0.5477465 ]]
~~~
These three numbers are what the model thinks are the probabilites that the flower is Iris-setosa, Iris-versicolor _or_ Iris-virginica, respectively.

Specifically, the model predicted there was a very small probabilty (0.0014901) that the sample flower species was Iris-setosa. However, it had considerable trouble deciding if the flower species was Iris-versicolor (0.45076337 or 45% chance) or Iris-virginica (0.5477465 or 55% chance). Going with the highest probability, the model "guessed" that it was Iris-virginica, but it still recorded that there was a 45% chance that real species was Iris-versicolor.   

### Example 7: Use Model to Make Two _ad hoc_ Predictions

You can also predict two sample flowers by stacking the sepal mesurements.
~~~text
# Specify the sepal length and width for the two Iris flower
sample_flowers = np.array( [[5.9,2.8],[5.1,3.5]],\
        dtype=float)
~~~
The code for making the actual predictions is the same:
~~~text
sepalPred = sepalModel.predict(sample_flowers)
~~~
Notice that the **argmax** in the second prediction requires **axis=1**.  Since we have a 2D array now, we must specify which axis to take the **argmax** over.  The value **axis=1** specifies we want the max column index for each row.
~~~text
sepalPred = np.argmax(sepalPred, axis=1)
~~~

In [13]:
# Example 7: Use the model to make two ad hoc predictions

# Specify the sepal length and width for the two Iris flower
sample_flowers = np.array( [[5.9,2.8],[5.1,3.5]],\
        dtype=float)

# Use the neural network to predict the species
sepalPred = sepalModel.predict(sample_flowers)

# Print out the results
print(sepalPred)
sepalPred = np.argmax(sepalPred, axis=1)
print(f"Model predicts sepal dimensions {sample_flowers} are from: {iris_species[sepalPred]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.20420103 0.3899119  0.40588704]
 [0.6252928  0.13561548 0.23909178]]
Model predicts sepal dimensions [[5.9 2.8]
 [5.1 3.5]] are from: Index(['Iris-virginica', 'Iris-setosa'], dtype='object')


If your code is correct you should see something similiar to the following output:
~~~text
1/1 [==============================] - 0s 21ms/step
[[0.0158558  0.5222274  0.46191677]
 [0.9817724  0.0140365  0.00419102]]
Model predicts sepal dimensions [[5.9 2.8]
 [5.1 3.5]] are from: Index(['Iris-versicolor', 'Iris-setosa'], dtype='object')
~~~

Looking at the output shown above, the `sepalModel` again had problems deciding between Iris-versicolor (prob=0.5222274 or 52%) and Iris-virginica (prob=0.46191677 or 46%) for the first flower, but was quite certain (prob=0.9817724 or 98%) that the second flower was from an Iris-setosa plant.

## **Exercise 1: Predict Iris Species**

For **Exercise 1** you are build the same neural network that was created in Example 1. Call your new neural network `petalModel`. The only real difference with your model will be the _features_ used to predict the species of an Iris flower. Rather than use sepal dimensions, you will train your model using **_petal_** dimensions (i.e. petal length and petal width).

### **Exercise 1-Step 1: Read datafile and create DataFrame**

In the cell below, write the code to read the CVS file, `iris.csv` from the course HTTPS server and store the information in a new DataFrame called `petalDF`. Shuffle and reindex the data, making sure to change the value of the random seed. Set your new `random seed = 30`.

**WARNING: If you don't change the value of the random seed to `30` you won't earn full credit for this lesson!**

After shuffling the data,  display the first 8 rows and columns to make sure the data was read correctly and your DataFrame was shuffled using the correct random seed.

In [ ]:
# Insert your code for Exercise 1-Step 1 here




If your code is correct, you should see the following table:

![___](https://biologicslab.co/BIO1173/images/class_03_3_Exe1.png)

If you table looks similar but has different numbers, it means you forgot to set the random seed = 30.

### **Exercise 1-Step 2: Create Feature Vector**

In the cell below create a feature vector for your neural network model.

You will need to modify the code used to generate your X-values. Call your X-values `petalX` and only use the data in the columns called `petal_length` and `petal_width`.

Since your feature vector will be used for classification, you will need to One-Hot Encode the column `species`. The code for doing this is exactly the same as the code in Example 1-Step 2 **_except_** you must call the array holding your Y-values `petalY`.

As always, make sure both your X-values and Y-values are in the correct, `float32` format.

Finally, print out the first 10 values in both the DataFrame `dummies` and your `petalY` array.

In [ ]:
# Insert your code for Exercise 1-Step 2 here



If your code is correct you should see the following output:

~~~text
     Iris-setosa  Iris-versicolor  Iris-virginica
20             1                0               0
5              1                0               0
3              1                0               0
101            0                0               1
..           ...              ...             ...
123            0                0               1
145            0                0               1
74             0                1               0
107            0                0               1

[10 rows x 3 columns]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
~~~

### **Exercise 1-Step 3: Construct, Compile and Train Neural Network**

In the cell below, write the code to build a neural network called `petalModel`. Since your new model will trained on different data, training will cause your model to have significantly different weights and biases between neurons.  

Make sure to set the input dimension to `petalX` as shown in this code chunk:
~~~text
petalModel.add(Input(shape=(petalX.shape[1],)))  # Input layer
~~~
Similarily, sure to set the output dimension to `petalY` as shown in this code chunk:
~~~text
petalModel.add(Dense(petalY.shape[1],activation='softmax')) # Output
~~~
As mentioned previously, when students in the course create neural network using "copy-and-paste", the often forget to change these values.

Finally, don't forget that you need to train your `petalModel` using the X-values in `petalX` and the Y-values in `petalY`.

In [ ]:
# Insert your code for Exercise 1-Step 3 here




If your code is correct you should see something similar to the following output:

~~~text
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 dense_5 (Dense)             (None, 50)                150       
                                                                 
 dense_6 (Dense)             (None, 25)                1275      
                                                                 
 dense_7 (Dense)             (None, 3)                 78        
                                                                 
=================================================================
Total params: 1,503
Trainable params: 1,503
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
5/5 - 0s - loss: 1.0804 - 265ms/epoch - 53ms/step
Epoch 2/100
5/5 - 0s - loss: 1.0275 - 28ms/epoch - 6ms/step
Epoch 3/100
5/5 - 0s - loss: 1.0020 - 37ms/epoch - 7ms/step
Epoch 4/100
5/5 - 0s - loss: 0.9877 - 27ms/epoch - 5ms/step
Epoch 5/100
5/5 - 0s - loss: 0.9729 - 27ms/epoch - 5ms/step

.......................

Epoch 95/100
5/5 - 0s - loss: 0.1660 - 18ms/epoch - 4ms/step
Epoch 96/100
5/5 - 0s - loss: 0.1647 - 19ms/epoch - 4ms/step
Epoch 97/100
5/5 - 0s - loss: 0.1629 - 20ms/epoch - 4ms/step
Epoch 98/100
5/5 - 0s - loss: 0.1606 - 19ms/epoch - 4ms/step
Epoch 99/100
5/5 - 0s - loss: 0.1629 - 20ms/epoch - 4ms/step
Epoch 100/100
5/5 - 0s - loss: 0.1607 - 19ms/epoch - 4ms/step

<keras.callbacks.History at 0x27e47fa9610>


### **Exercise 2: Print Predictions**

In the cell below compute the predictions made by your `petalModel` and store these values in a new variable called `petalPred`. Print out the predictions for the first 10 Iris flowers.

In [ ]:
# Insert your code for Exercise 2 here



If your code is correct you should see something similar to the following:

~~~text
5/5 [==============================] - 0s 2ms/step
Shape of petalPred: (150, 3)
[[0.99420524 0.00576605 0.00002864]
 [0.98149616 0.01836091 0.000143  ]
 [0.9948488  0.00511919 0.00003205]
 [0.00001897 0.14643341 0.85354763]
 [0.00697531 0.9362573  0.05676734]
 [0.01162005 0.9580431  0.03033691]
 [0.0000592  0.24554215 0.75439864]
 [0.00000036 0.01080822 0.9891914 ]
 [0.00268251 0.90185    0.09546751]
 [0.0000111  0.36802647 0.6319624 ]]
~~~

### **Exercise 3:  Print Predicted and Expected Values**

In the cell below, print the predicted and the actual values (class) for your `petalModel`.

To do this you need to find the maximum prediction for each row using this code chunk:
~~~text
# Find the maximum prediction for each row
petalPredict_classes = np.argmax(petalPred,axis=1)
~~~
You will also have to find the expected value for each row using this code chunk:
~~~text
# Find the expected value for each row
petalExpected_classes = np.argmax(petalY,axis=1)
~~~

Once you have determined these values, you can print the results with this code chunk:
~~~text
# Print out the results
print(f"Predictions: {petalPredict_classes}")
print(f"Expected: {petalExpected_classes}")
~~~~

In [ ]:
# Insert your code for Exercise 3 here



If your code is correct you should see the following output:

~~~text
Predictions: [0 0 0 2 1 1 2 2 1 2 0 2 1 1 0 1 0 0 0 1 1 0 0 0 2 2 2 2 0 1 2 1 2 2 2 2 1
 2 1 1 2 2 0 1 2 1 1 1 1 1 0 1 2 1 0 1 0 1 1 1 0 1 0 1 0 2 2 2 0 2 2 2 0 0
 1 0 2 2 2 2 0 1 0 1 1 1 2 0 1 0 1 2 1 0 0 0 2 2 0 1 1 1 0 0 0 0 2 0 2 0 0
 2 2 2 0 2 1 1 2 1 1 1 1 2 0 1 0 1 0 0 0 2 1 0 0 1 1 2 2 2 0 1 1 0 2 1 2 2
 0 0]
Expected: [0 0 0 2 1 1 2 2 1 2 0 2 1 1 0 1 0 0 0 1 2 0 0 0 2 2 1 2 0 1 2 1 2 2 2 2 1
 2 1 2 2 2 0 1 2 1 1 1 1 1 0 1 2 1 0 1 0 1 1 1 0 1 0 1 0 2 2 2 0 2 1 2 0 0
 1 0 2 2 2 2 0 1 0 1 1 1 2 0 1 0 1 2 1 0 0 0 2 2 0 1 1 1 0 0 0 0 2 0 2 0 0
 2 2 2 0 2 1 1 2 1 1 1 1 2 0 1 0 1 0 0 0 2 1 0 0 1 2 2 2 2 0 1 2 0 2 1 2 2
 0 0]
~~~

This output shows your model's prediction of each flower's species (top array) as well as their actual (expected) species name (bottom array). The number `0` means Iris-setosa, `1` means Iris-versicolor and `2` means Iris-virginica. Just by a simple, visual comparison, you can see that your model did a very good job predicting a flower's species based on its petal dimensions.


### **Exercise 4: Convert Index Values into Species Names**

In the cell below, print out the first 5 values in `petalPredict_classes` and then using the index variable `iris_species`, print the species names of these first 5 flowers.


In [ ]:
# Insert your code for Exercise 4 here



If your code is correct you should see:
~~~text
petalPredict_classes: [0 0 0 2]
Iris-setosa Iris-setosa Iris-setosa Iris-virginica
~~~


### **Exercise 5: Compute the Accuracy Score**

In the cell below, compute the accuracy score for your `petalModel` neural network model. Use the f-sting print statement to print the accuracy score.


In [ ]:
# Insert your code for Exericse 5 here



If your code is correct, you should see the following output:
~~~text
Accuracy: 0.96
~~~
WOW! Your `petalModel` is roughly 95% accurate.

This accuracy for your `petalModel` is substantially higher than the accuracy of `sepalModel` created in Example 1.

Since the same neural network architecture was used for both models, this difference in accuracy must mean that **_petal_** dimensions are more different in these 3 Iris species than their **_sepal_** dimensions.

You can verify this fact by comparing the mean dimensions for sepal and petals shown below in the Appendix.

### **Exercise 6: Use the Model to Make an _ad hoc_ Prediction**

Use your `petalModel` to make an _ad hoc_ prediction of an unknown Iris species. You measured the petal length to be 1.5 cm and a petal width of 0.3 cm. Print out the model's prediction.

In [ ]:
# Insert your code for Exercise 6 here



If your code is correct you should see something similar to the following:
~~~text
1/1 [==============================] - 0s 19ms/step
[[0.9773777  0.02249157 0.00013073]]
Model predicts flower with petal dimensions [[1.5 0.3]] is: Iris-setosa
~~~


### **Exercise 7: Use Model to Make Two _ad hoc_ Predictions**

Use your `petalModel` to make 2 _ad hoc_ predictions. The petal dimensions are 4.2 and 1.3 cm for the first flower and 5.1 and 3.5 cm for the second flower. As in Exercise 6, print out the model's predictions as probabilies and then the two highest probabilites as the model's final choices.

In [ ]:
# Insert your code for Exercise 7 here



If your code is correct you should see something similar to the following:
~~~text
1/1 [==============================] - 0s 22ms/step
[[0.00969335 0.7415197  0.24878693]
 [0.00000006 0.00312731 0.99687254]]
Model predicts flowers with petal dimensions [[4.2 1.3]
 [5.1 3.5]] are: Index(['Iris-versicolor', 'Iris-virginica'], dtype='object')
~~~

## **Lesson Turn-in**

When you have completed and run all of the code cells, use the **File --> Print.. --> Save to PDF** to generate a PDF of your Colab notebook. Save your PDF as `Class_03_3.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.

---------------------------------------------

### **Appendix**

One of the more interesting results of this lesson was the ability of a very simple neural network to more accurately predict a flower's species when it had been trained on petal dimensions compared to sepal dimensions. Specifically, the accuracy of your `petalModel` was close to 95% while the `sepalModel` was only about 75% accurate.

Since these two neural networks had exactly the same architecture, and were both trained for 100 epochs, the difference in accuracy must be attributable to species specific differences in sepal and petal dimensions. In other words, the **_range_** in petal dimensions must be more pronounced in these 3 Iris species than their sepal dimensions.  

In the cells below, the Pandas `df.groupby()` function was used to compute the mean values of sepal length, sepal width, petal width, and petal width in the three different Iris flower species, _I. setosa_, _I. versicolor_, and _I. virginica_.

As expected, species differences for the mean petal dimensions are much greater than the mean sepal dimensions. These differences are sufficient to explain why the accuracy of your `petalModel` was so much higher (\~95% accurate) than the `sepalModel` (\~75% accurate).


In [ ]:
# Group Iris flower species by mean Sepal Length
print("Mean SEPAL LENGTHS by species")
sepal_length = petalDF.groupby('species')['sepal_length'].mean()
sepal_length

In [ ]:
# Group Iris flower species by mean Sepal Width
print("Mean SEPAL WIDTHS by species")
sepal_width = petalDF.groupby('species')['sepal_width'].mean()
sepal_width

In [ ]:
# Group Iris flower species by mean Petal Length
print("Mean PETAL LENGTHS by species")
petal_length = petalDF.groupby('species')['petal_length'].mean()
petal_length

In [ ]:
# Group Iris flower species by mean Petal Width
print("Mean PETAL WIDTHS by species")
petal_width = petalDF.groupby('species')['petal_width'].mean()
petal_width